In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Copy from google drive to google collab
# Change path
!cp -r "/content/drive/MyDrive/Test/Processed/Electronics.zip" "/content/"

In [ ]:
# Copy from google drive to google collab
# Change path
#!cp -r "/content/drive/MyDrive/Test/Processed/Movies_and_TV.zip" "/content/"

In [ ]:
# Copy from google drive to google collab
# Change path
#!cp -r "/content/drive/MyDrive/Test/Processed/CDs_and_Vinyl.zip" "/content/"

In [ ]:
# Copy from google drive to google collab
# Change path
#!cp -r "/content/drive/MyDrive/Test/Processed/Books.zip" "/content/"

Unzip into google collab drive

In [ ]:
#!unzip Movies_and_TV.zip -d '/content/'

In [ ]:
#!unzip CDs_and_Vinyl.zip -d '/content/'

In [ ]:
#!unzip Books.zip -d '/content/'

In [ ]:
!unzip Electronics.zip -d '/content/'

Archive:  Electronics.zip
  inflating: /content/content/Electronics.csv  


In [ ]:
!pip install -q contractions
!pip install -q pyspellchecker
!pip install -q lexical_diversity
!pip install -q jaccard_index
!pip install -q distance
!pip install -q swifter

In [ ]:
# Category to process: CD, BOOK, ELEC, MOVIE
DATA_CATEGORY = 'ELEC'
SAVE_FEATURE_ENGINEERING = True
DATA_SPLIT = False

if DATA_CATEGORY == 'CD':

    DATA_NAME = 'CDs_and_Vinyl'

elif DATA_CATEGORY == 'BOOK':

    DATA_NAME = 'Books'

elif DATA_CATEGORY == 'ELEC':

    DATA_NAME = 'Electronics'

elif DATA_CATEGORY == 'MOVIE':

    DATA_NAME = 'Movies_and_TV'

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag
from bs4 import BeautifulSoup

import unicodedata
import contractions
import numpy as np
import re
from textblob import TextBlob
# Need to install via pyspellchecker
from spellchecker import SpellChecker
from lexical_diversity import lex_div as ld
from jaccard_index.jaccard import jaccard_index
import distance
import string
import html as ihtml

from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd

import multiprocessing
import swifter

import sys
# initialization
spell = SpellChecker()

stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)

In [ ]:
path = '/content/' + DATA_NAME +'.csv'

df = pd.read_csv(path)

In [ ]:
df.info()

# Data preprocessing and cleaning

Note: Vote > 10 has been execute in the JSON Preprocessing

In [ ]:
def strip_html(text):
    text = str(text)
    text = BeautifulSoup(ihtml.unescape(text), "lxml").text
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"http+", "", text)
    text = re.sub(r"http[s]?://\S+", "", text)
    text = re.sub(r"\s+", " ", text)

    return text

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

# Define function to expand contractions
def expand_contractions(text):
    return(contractions.fix(text))

# special_characters removal
def remove_special_characters(text):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, ' ', text)
    return text

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words


def remove_punctuation_and_splchars(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_word = remove_special_characters(new_word)
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords_list:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation_and_splchars(words)
    words = remove_stopwords(words)
    return words

def lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

def clean_text(input):
    sample = denoise_text(input)
    sample = expand_contractions(sample)
    sample = remove_special_characters(sample)

    return sample

def normalize_and_lemmaize(input):
    sample = denoise_text(input)
    sample = expand_contractions(sample)
    sample = remove_special_characters(sample)
    words = nltk.word_tokenize(sample)
    words = normalize(words)
    lemmas = lemmatize(words)

    return ' '.join(lemmas)

In [ ]:
# Calculate number of tagging: noun, adjective, etc.

def get_number_of_words(text):
  return len(text)

def get_number_of_sentences(text):
  sentences= nltk.sent_tokenize(text)
  return len(sentences)

def get_number_of_character(text):
  return len(text)

def get_number_of_question(text):
  return len(text.split('? '))

def get_number_of_exclamation(text):
  return len(text.split('! '))

def get_pos_tagging(text):
  return([pos_tag[1] for pos_tag in pos_tag(text)])

def get_number_of_upper(text):
  return len([char for char in text if char.isupper()==True]) 

def get_polarity(text):
  return TextBlob(text).sentiment.polarity

def get_subjectivity(text):
  return TextBlob(text).sentiment.subjectivity

def get_number_of_misspell(tokens):
  return len(spell.unknown(tokens))

def get_number_of_noun(pos_tag):
  return sum(1 for pos in pos_tag if pos.startswith('NN'))

def get_number_of_adjective(pos_tag):
  return sum(1 for pos in pos_tag if pos.startswith('JJ'))

def get_number_of_adverb(pos_tag):
  return sum(1 for pos in pos_tag if pos.startswith('RB'))

def get_number_of_verb(pos_tag):
  return sum(1 for pos in pos_tag if pos.startswith('VB'))

def get_lex_diversity(token):
  return(ld.ttr(token)) 

def get_num_one_letter(token):
  return sum(1 for i in token if len(i) == 1)

def get_num_two_letter(token):
  return sum(1 for i in token if len(i) == 2)

def get_num_long_letter(token):
  return sum(1 for i in token if len(i) >= 3)

def get_num_unique_word(token):
  unique = set(token)
  return len(unique)

def get_num_stopword(token):
  num_stopwords = [w for w in token if w in stopwords_list]

  return len(num_stopwords)

def get_cosine_similarity(title, cleanReview):
  if len(cleanReview) == 0:
    return 0
  return 1 - distance.jaccard(title,cleanReview)

def get_deviation_extreme(rate, min, max):
  abs_min = abs(rate-min)
  abs_max = abs(rate-max)

  if (abs_min >= abs_max):
    return abs_min
  elif (abs_min < abs_max):
    return abs_max

def get_helpfulness(positive,totalVote,helpful,unhelpful):

  if(positive/totalVote > helpful):
    return 1
  elif(positive/totalVote < unhelpful):
    return 0

# Check Nan and set as zero: reviewer_std_rating, product_std_rating


In [ ]:
# Before dropping data with NaN
df.info()

In [ ]:
print('Total NaN in DataFrame', df.isnull().sum().sum())

df.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

# Drop unwated column (Unnamed: 0)
df = df.drop(df.columns[0],axis=1)

df.info()

In [ ]:
def word_sampling(df, n_sample, n_word, random_state):

  df_subset = df.sample(n=n_sample, random_state = random_state)

  df_subset['reviewText']=df_subset['reviewText'].apply(str)

  df_subset['ori_cleaned_review'] = \
  df_subset['reviewText'].swifter.progress_bar(False).apply(lambda x:  \
                                                            clean_text(x))

  df_subset['ori_cleaned_tokens'] =  \
    df_subset['ori_cleaned_review'].swifter.progress_bar(False).apply(lambda x: \
                                                        nltk.word_tokenize(x))

  df_subset['word'] = \
    df_subset['ori_cleaned_tokens'].swifter.progress_bar(False).apply(lambda x:\
                                                          get_number_of_words(x))
  # review without word will be filtered.  
  df_subset = df_subset[df_subset['word'] > 0]

  df_to_remove = df_subset[df_subset['word'] > n_word]

  df_subset = df_subset.drop(df_to_remove.index)

  n_standing = 0

  while len(df_to_remove.index) != 0:

    df = df.drop(df_to_remove.index)

    n_outstanding = n_sample - len(df_subset.index)

    df_outstanding = df.sample(n_outstanding, random_state = random_state)

    df_outstanding['ori_cleaned_review'] = \
      df_outstanding['reviewText'].swifter.progress_bar(False).apply(lambda x:  clean_text(x))

    df_outstanding['ori_cleaned_tokens'] =  \
      df_outstanding['ori_cleaned_review'].swifter.progress_bar(False).apply(lambda \
                                                      x:  nltk.word_tokenize(x))

    df_outstanding['word'] = \
      df_outstanding['ori_cleaned_tokens'].swifter.progress_bar(False).apply(lambda x: \
                                                          get_number_of_words(x))

    df_to_remove = df_outstanding[df_outstanding['word'] > n_word]

    df_outstanding = df_outstanding.drop(df_to_remove.index)

    df_subset = pd.concat([df_subset, df_outstanding])

  return df_subset


Data is order based on reviewer id therefore shuffling to make sure all reviewer, review and product can be covered. 

In [ ]:
df = df.sample(frac=1,random_state = 42).reset_index(drop=True)

# Data Split

In [ ]:
# #Process 75 % ratio first

if DATA_SPLIT == True:
  df_75 = word_sampling(df = df[df['helpful_ratio'] > 0.75], n_sample = 50000, \
                      n_word = 500, random_state = 42)

  df_35 = word_sampling(df = df[df['helpful_ratio'] < 0.35], n_sample = 50000, \
                      n_word = 500, random_state = 42)

  #save memory
  del df

  df = pd.concat([df_75,df_35])

  #save memory
  del df_75, df_35

  df = df.reset_index(drop=True)

  df.info()

  df.describe()

# Feature Engineering

In [ ]:
if DATA_SPLIT == False:

  df['reviewText']=df['reviewText'].apply(str)

  df['ori_cleaned_review'] = \
      df['reviewText'].swifter.progress_bar(True).apply(lambda x:  \
                                                            clean_text(x))

  df['ori_cleaned_tokens'] =  \
      df['ori_cleaned_review'].swifter.progress_bar(True).apply(lambda x: \
                                                        nltk.word_tokenize(x))

  df['word'] = \
      df['ori_cleaned_tokens'].swifter.progress_bar(True).apply(lambda x:\
                                                          get_number_of_words(x))

  #Take only comment with maximum length 500 words, and more than 0 word    
  df = df[(df['word'] <= 500) & (df['word'] > 0)]

  df['char']    = df['ori_cleaned_review'].swifter.apply(lambda x: get_number_of_character(x))

  df = df.drop('ori_cleaned_review',axis=1)

In [ ]:
# structual features
# 3 minutes run for 40k data

# Prepare token

df['original_tokens'] = df['reviewText'].swifter.apply(lambda x:  nltk.word_tokenize(x))

# Original token processing  ======================= start
df['spell'] = df['original_tokens'].swifter.apply(lambda x: get_number_of_misspell(x))
# save memory space
df = df.drop('original_tokens',axis=1)
# Original token processing  ======================= end

# original cleaned token  ======================= start
# clean review

df['num_one_letter'] = df['ori_cleaned_tokens'].swifter.apply(lambda x:  get_num_one_letter(x))


df['num_two_letter'] = df['ori_cleaned_tokens'].swifter.apply(lambda x:  get_num_two_letter(x))

df['num_long_letter'] = df['ori_cleaned_tokens'].swifter.apply(lambda x:  get_num_long_letter(x))

df['num_stop_word'] = df['ori_cleaned_tokens'].swifter.apply(lambda x: get_num_stopword(x))
# save memory space
df = df.drop('ori_cleaned_tokens',axis=1)
# original cleaned token  ======================= end

# Normal review text

df['sent']    = df['reviewText'].swifter.apply(lambda x: get_number_of_sentences(x))

df['question'] = df['reviewText'].swifter.apply(lambda x: get_number_of_question(x))

df['exclam']  = df['reviewText'].swifter.apply(lambda x: get_number_of_exclamation(x))

df['caps']    = df['reviewText'].swifter.apply(lambda x: get_number_of_upper(x))

# calculation
df['len']     = df['char'] / df['word']
df['avg']     = df['word'] / df['sent']
df['per']     = df['question'] / df['sent']


In [ ]:
df['norm_lem_Text'] = df['reviewText'].swifter.apply(lambda text: normalize_and_lemmaize(text))

# norm_lem_Text processing ======================= start
# Sentiment feature engineering

df['polarity'] = df['norm_lem_Text'].swifter.apply(lambda text: get_polarity(text))

df['subjectivity'] = df['norm_lem_Text'].swifter.apply(lambda text: get_subjectivity(text))

df['norm_lem_tokens'] = df['norm_lem_Text'].swifter.apply(lambda text: nltk.word_tokenize(text))
# norm_lem_Text processing ======================= end 

# save memory space
df = df.drop('norm_lem_Text',axis=1)

# norm_lem_token processing =========================== start

df['pos'] = df['norm_lem_tokens'].swifter.apply(lambda text: get_pos_tagging(text))

df['unique_word'] = df['norm_lem_tokens'].swifter.apply(lambda x: get_num_unique_word(x))

# norm_lem_token processing =========================== end

# POS processing ====================================== start

df['pos_len'] = df['pos'].swifter.apply(lambda x: len(x))
# Lexicol structural feature engineering

df['noun'] = df['pos'].swifter.apply(lambda text: get_number_of_noun(text))

df['noun_ratio'] =  df['noun'] / df['pos_len']

df['adjective'] = df['pos'].swifter.apply(lambda text: get_number_of_adjective(text))

df['adverb'] = df['pos'].swifter.apply(lambda text: get_number_of_adverb(text))

df['adverb_ratio'] = df['adverb'] / df['pos_len']

df['verb'] = df['pos'].swifter.apply(lambda text: get_number_of_verb(text))

df['verb_ratio'] = df['verb'] / df['pos_len']

df['lex_diversity'] =  df['pos'].swifter.apply(lambda text: get_lex_diversity(text))

# save memory space
df = df.drop('pos',axis=1)

# To resolve 0 division issue

# POS processing ====================================== end

# Title/Summary Feature

In [ ]:
# normalization and lemmaize

df['summary']=df['summary'].apply(str)

df['cleanSummary'] = df['summary'].swifter.apply(lambda text: normalize_and_lemmaize(text))

# tokenize the summary/title

df['clean_summary_tokens'] = df['cleanSummary'].swifter.apply(lambda text: nltk.word_tokenize(text))

# feature engineering for summary/title

df['title_cosine_similarity'] = df.swifter.apply(lambda x : get_cosine_similarity( x['clean_summary_tokens'],x['norm_lem_tokens']),axis=1)

# norm_lem_tokens- save ram space
df = df.drop('norm_lem_tokens',axis=1)

df['title_Polarity'] = df['summary'].swifter.apply(lambda text: get_polarity(text))

df['title_subjectivity'] = df['summary'].swifter.apply(lambda text: get_subjectivity(text))

df['title_word']    = df['summary'].swifter.apply(lambda x: get_number_of_words(x))

# save memory space
df = df.drop(['cleanSummary', 'clean_summary_tokens'],axis=1)

In [ ]:
# To resolve NaN issue due to 0 devision
df.replace([np.inf, -np.inf, np.nan], 0, inplace=True)

# Label: Helpful or Unhelpful

In [ ]:
df = df[df['total'] > 0]

In [ ]:
df['helpfulness'] = df.swifter.apply(lambda x : get_helpfulness(x['positive'],\
                                  x['total'], helpful = 0.75, \
                                  unhelpful = 0.35),axis=1).astype("category")

# Save Data for LIWC processing

In [ ]:
df.iloc[:,0:20]

In [ ]:
# Save the result from JSON preprocessing result
# as csv to google collab

df = df.reset_index(drop=True)

path = '/content/' + DATA_NAME + '_Almasdi_Processed.csv'

with open(path, 'w', encoding = 'utf-8-sig') as f:
    df.to_csv(f)

In [ ]:
# zip and save in google drive
# change path name
#!zip -r '/content/drive/MyDrive/Test/Processed/Movies_and_TV_Almasdi_Processed.zip' '/content/Movies_and_TV_Almasdi_Processed.csv'

In [ ]:
# zip and save in google drive
# change path name
!zip -r '/content/drive/MyDrive/Test/Processed/CDs_and_Vinyl_Almasdi_Processed.zip' '/content/CDs_and_Vinyl_Almasdi_Processed.csv'

In [ ]:
# zip and save in google drive
# change path name
#!zip -r '/content/drive/MyDrive/Test/Processed/Electronics_Almasdi_Processed.zip' '/content/Electronics_Almasdi_Processed.csv'

In [ ]:
# zip and save in google drive
# change path name
#!zip -r '/content/drive/MyDrive/Test/Processed/Books_Almasdi_Processed.zip' '/content/Books_Almasdi_Processed.csv'